# Test ReCAPTCHA de Google


Dans cette partie nous utiliser les ressource présent dans le répertoir "test"

In [1]:
# import de libraire

import matplotlib.pyplot as plt
import PIL
from PIL import Image
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import urllib2
import os
import cookielib
import json
import time
from collections import Counter

## Découpage de l'image

l'image du test de google et en fait une seul image qu'il faut découpé découpé (içi 4 x 4)

In [88]:
xDiv = 4
yDiv = 4
filename = 'test/test panneau 4x4.jpeg'


img = Image.open(filename)
x, y = img.size
xsize = x/xDiv
ysize = y/yDiv

for x in range(xDiv):
    for y in range(yDiv):
        imgpiece = img.crop((x*xsize, y*ysize, (x+1)*xsize, (y+1)*ysize))
        #sauve sous le nom img_x_y.png, toto_0_1.jpeg
        imgpiece.save('images_decoup/img' + '_' + str(x) + '_' + str(y) + '.jpg')    


## Partie pour le téléchargement d'images

le but et de télécharger un grand nombre d'images sur google images avec un mot clé définie 
(içi : panneaux de signalisation)

In [96]:
search = "panneaux de signalisation"

def get_soup(url,header):
    return BeautifulSoup(urllib2.urlopen(urllib2.Request(url,headers=header)),'html.parser')



def download(): 
        query= search.split()
        query='+'.join(query)
        url="https://www.google.fr/search?q="+query+"&source=lnms&tbm=isch&tbs=ic:specific"
        print url
        #add the directory for your image here
        DIR="img"
        header={'User-Agent':"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36"
        }
        soup = get_soup(url,header) #by stack


        ActualImages=[]
        for a in soup.find_all("div",{"class":"rg_meta"}):
            link , Type =json.loads(a.text)["ou"]  ,json.loads(a.text)["ity"]
            ActualImages.append((link,Type))

        print  "there are total" , len(ActualImages),"images"

        if not os.path.exists(DIR):
                    os.mkdir(DIR)
        if not os.path.exists(DIR):
                    os.mkdir(DIR)
        cntr = 0
        for i , (img , Type) in enumerate( ActualImages):
            try:
                req = urllib2.Request(img, headers={'User-Agent' : header})
                raw_img = urllib2.urlopen(req).read()
                if len(Type)==3:
                    cntr = cntr + 1
                    f = open(os.path.join(DIR ,"big"+"Img"+"_"+ str(cntr)+".jpg"), 'wb')
                    f.write(raw_img)
                    f.close()
                    size = xsize,ysize
                    im = Image.open("img/big"+"Img"+"_"+ str(cntr)+".jpg")
                    im.thumbnail(size, Image.ANTIALIAS)
                    im.save("img/"+"Img"+"_"+ str(cntr)+".jpg", "JPEG")



            except Exception as e:
                print "could not load : "+img
                print e
        print "Download finished"

        
download()

https://www.google.fr/search?q=panneaux+de+signalisation&source=lnms&tbm=isch&tbs=ic:specific
there are total 100 images
could not load : http://applications.eduscol.education.fr/D0160/03_034assr12signalisation_pan3.gif
cannot write mode P as JPEG
could not load : http://applications.eduscol.education.fr/D0160/03_034assr12signalisation_pan5.gif
cannot write mode P as JPEG
could not load : http://www.hellopro.fr/images/produit-2/7/0/2/panneau-de-signalisation-c107-1299207.gif
cannot write mode P as JPEG
could not load : https://www.coderouteplus.com/wp-content/uploads/2015/11/panneaux-de-signalisation.jpg
<urlopen error [Errno -2] Name or service not known>
could not load : http://www.expression-anglaise.com/wp-content/uploads/2015/04/interdiction-demi-tour.gif
cannot write mode P as JPEG
could not load : http://www.hellopro.fr/images/produit-2/2/5/0/panneau-de-signalisation-b2b-1299052.gif
cannot write mode P as JPEG
could not load : http://www.hellopro.fr/images/produit-2/4/9/0/pannea

## Classification d'immages

Le but de cette dernier partie et de dire si oui ou non les images découpé ce raproche d'un panneaux de signalisation avec une aproximation

### répertorie les pixels

répertorie les pixels des images de références dans un fichier text (pour le faire qu'une fois) 

In [97]:
  
numberPerType = range(1,99)


for furtherNum in numberPerType:
    try:
        numberArrayExamples = open('numAr.txt','a')
        imgFilePath = 'img/Img_'+str(furtherNum)+'.jpg'
        ei = Image.open(imgFilePath)      
        eiar = np.array(ei)
        eiarl = str(eiar.tolist())
        lineToWrite = str(furtherNum)+'::'+eiarl+'\n'
        numberArrayExamples.write(lineToWrite)
        numberArrayExamples.close()
        ei.close()
        del ei
        del eiar
        del eiarl
    except Exception as e:
        print e


cannot identify image file 'img/Img_2.jpg'
cannot identify image file 'img/Img_4.jpg'
cannot identify image file 'img/Img_9.jpg'
cannot identify image file 'img/Img_14.jpg'
cannot identify image file 'img/Img_18.jpg'
cannot identify image file 'img/Img_22.jpg'
cannot identify image file 'img/Img_25.jpg'
cannot identify image file 'img/Img_29.jpg'
cannot identify image file 'img/Img_51.jpg'
cannot identify image file 'img/Img_61.jpg'
[Errno 2] No such file or directory: 'img/Img_66.jpg'
cannot identify image file 'img/Img_67.jpg'
cannot identify image file 'img/Img_92.jpg'
cannot identify image file 'img/Img_94.jpg'
[Errno 2] No such file or directory: 'img/Img_95.jpg'
[Errno 2] No such file or directory: 'img/Img_96.jpg'
[Errno 2] No such file or directory: 'img/Img_97.jpg'
[Errno 2] No such file or directory: 'img/Img_98.jpg'


### Compare

compare le nombre de pixel en commun avec les images de références

In [98]:
def whatNumIsImg(filePath):

    matchedAr = []
    loadExamps = open('numAr.txt','r').read()
    loadExamps = loadExamps.split('\n')
    
    i = Image.open(filePath)
    iar = np.array(i)
    iarl = iar.tolist()

    inQuestion = str(iarl)

    for eachExample in loadExamps:
        try:
            splitEx = eachExample.split('::')
            currentNum = splitEx[0]
            currentAr = splitEx[1]
            
            eachPixEx = currentAr.split('],')
            eachPixInQ = inQuestion.split('],')

            x = 0

            while x < len(eachPixEx):
                if eachPixEx[x] == eachPixInQ[x]:
                    matchedAr.append(int(currentNum))

                x+=1
        except Exception as e:
            e
            
    print filePath + ' = ' + str(len(matchedAr))

    
for x in range(xDiv):
    for y in range(yDiv):
        whatNumIsImg('images_decoup/img_'+str(x)+'_'+str(y)+'.jpg')

images_decoup/img_0_0.jpg = 2
images_decoup/img_0_1.jpg = 8
images_decoup/img_0_2.jpg = 2
images_decoup/img_0_3.jpg = 5
images_decoup/img_1_0.jpg = 3
images_decoup/img_1_1.jpg = 4
images_decoup/img_1_2.jpg = 3
images_decoup/img_1_3.jpg = 3
images_decoup/img_2_0.jpg = 1
images_decoup/img_2_1.jpg = 2
images_decoup/img_2_2.jpg = 0
images_decoup/img_2_3.jpg = 5
images_decoup/img_3_0.jpg = 3
images_decoup/img_3_1.jpg = 7
images_decoup/img_3_2.jpg = 2
images_decoup/img_3_3.jpg = 4


# Conclusion

la comparéson na pas du tout marcher les bonnes images a trouver son :
    mg_0_2.jpg
    mg_1_2.jpg
    mg_2_2.jpg
    mg_3_2.jpg